In [9]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import re
def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

In [2]:
# Cities to search
cities = ['Los-Angeles_CA','New-York_NY']
#  , 'Chicago_IL', 'Houston_TX', 'Phoenix_AZ', 'Philadelphia_PA', 'San-Antonio_TX', 'San-Diego_CA', 'Dallas_TX', 'San-Jose_CA']

In [3]:
# Start browser
browser = init_browser()



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/kevingil/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


In [4]:
# Create blank lists to store attributes
prices = []
beds = []
baths = []
sizes = []
addresses = []
statuses = []
pages = []
types = []
years = []

In [18]:
# Loop through each city
for city in cities:

    # Loop through each search result page
    for i in range(1, 5, 1):

        # Set variables
        url = f"https://www.realtor.com/realestateandhomes-search/{city}/pg-{i}"
        browser.visit(url)
        
        print('LOOP START')
        
        # HTML object
        html = browser.html
        
        print('Visiting the URL')
        print(f"\nScraping page: {i}")
        
        # Parse HTML with Beautiful Soup
        soup = bs(html, "html.parser")

        # Identify all listings
        listings = soup.find_all('li', attrs={"data-testid": "result-card"})

        # Loop through each listing
        for listing in listings:          
            try:
                price = listing.find('span', attrs={"data-label": "pc-price"}).text
                
                # Convert to integer
                price = price.replace('[\$\,\.]',"",regex=True).astype(int)
                prices.append(price)
                
                print(price)
            except:
                prices.append('No Info')
                print('No Info')
            try:
                bed = listing.find('li', attrs={"data-label": "pc-meta-beds"}).text
                beds.append(bed)
                print(bed)
            except:
                beds.append('No Info')
                print('No Info')
            try:
                bath = listing.find('li', attrs={"data-label": "pc-meta-baths"}).text
                baths.append(bath)
                print(bath)
            except:
                baths.append('No Info')
                print('No Info')
            try:
                size = listing.find('li', attrs={"data-label": "pc-meta-sqft"}).text
                sizes.append(size)
                print(size)
            except:
                sizes.append('No Info')
                print('No Info')
            try:
                address = listing.find('div', attrs={"data-label": "pc-address"}).text
                addresses.append(address)
                print(address)
            except:
                addresses.append('No Info')
                print('No Info')
            try:
                status = listing.find('span', attrs={"class": "jsx-3853574337 statusText"}).text
                statuses.append(status)
                print(status)
            except:
                statuses.append('No Info')
                print('No Info')
            print("----------------------------")

            
            
            
            

#             price = listing.find('span', attrs={"data-label": "pc-price"})
#             bed = listing.find('li', attrs={"data-label": "pc-meta-beds"})
#             bath = listing.find('li', attrs={"data-label": "pc-meta-baths"})
#             size = listing.find('li', attrs={"data-label": "pc-meta-sqft"})
#             address = listing.find('div', attrs={"data-label": "pc-address"})
#             status = listing.find('span', attrs={"class": "jsx-3853574337 statusText"})
                    

#             if price and price.text:
#                 prices.append(price.text)
#             else: 
#                 prices.append('No Info')

#             if bed and bed.text:
#                 beds.append(bed.text)
#             else: 
#                 beds.append('No Info')

#             if bath and bath.text:
#                 baths.append(bath.text)
#             else: 
#                 baths.append('No Info')

#             if size and size.text:
#                 sizes.append(size.text)
#             else: 
#                 sizes.append('No Info')

#             if address and address.text:
#                 addresses.append(address.text)
#             else: 
#                 addresses.append('No Info')

#             if status and status.text:
#                 statuses.append(status.text)
#             else: 
#                 statuses.append('No Info')
                
            
            # Identify URL to listing detail page
            page = listing.find('a').get('href')
            
            # Append to list
            pages.append(page)

LOOP START
Visiting the URL

Scraping page: 1
No Info
4bed
3bath
1,740sqft
1342 W 79th St, Los Angeles, CA 90044
For Sale
----------------------------
No Info
5bed
No Info
No Info
1130 E 89th St, Los Angeles, CA 90002
For Sale
----------------------------
No Info
3bed
2.5bath
2,053sqft
12105 Via Santa Marta, Sylmar, CA 91342
For Sale
----------------------------
No Info
3bed
2bath
1,500sqft
21500 Lassen St Spc 98, Chatsworth, CA 91311
For Sale
----------------------------
No Info
3bed
2bath
1,501sqft
13970 Olive Grove Ln, Sylmar, CA 91342
For Sale
----------------------------
No Info
3bed
3bath
1,682sqft
9315 Burnet Ave Unit 112, North Hills, CA 91343
For Sale
----------------------------
No Info
3bed
2bath
1,507sqft
13216 S Main St, Los Angeles, CA 90061
For Sale
----------------------------
No Info
No Info
5bath
2,300sqft
3536 W Slauson Ave, Los Angeles, CA 90043
For Sale
----------------------------
No Info
4bed
4bath
2,946sqft
17323 San Jose St, Granada Hills, CA 91344
For Sale
---

LOOP START
Visiting the URL

Scraping page: 3
No Info
2bed
1bath
1,072sqft
1616 W 24th St, Los Angeles, CA 90007
For Sale
----------------------------
No Info
2bed
1bath
716sqft
208 W 98th St, Los Angeles, CA 90003
For Sale
----------------------------
No Info
5bed
2bath
1,796sqft
1702 W 57th St, Los Angeles, CA 90062
For Sale
----------------------------
No Info
Studio
1bath
552sqft
620 S Gramercy Pl Apt 423, Los Angeles, CA 90005
For Sale
----------------------------
No Info
4bed
2bath
1,526sqft
434 E 90th St, Los Angeles, CA 90003
For Sale
----------------------------
No Info
4bed
2bath
1,866sqft
5132 LA Calandria Way, Los Angeles, CA 90032
For Sale
----------------------------
No Info
3bed
3bath
1,600sqft
3243 W 60th St, Los Angeles, CA 90043
For Sale
----------------------------
No Info
3bed
1.5bath
1,246sqft
4228 7th Ave, Los Angeles, CA 90008
For Sale
----------------------------
No Info
2bed
1bath
1,196sqft
8529 Mammoth Ave, Panorama City, CA 91402
For Sale
--------------------

LOOP START
Visiting the URL

Scraping page: 1
No Info
6bed
2.5bath
No Info
17 Erwin Ct, Staten Island, NY 10306
For Sale
----------------------------
No Info
3bed
2.5bath
1,620sqft
270 Blair Ave, Bronx, NY 10465
For Sale
----------------------------
No Info
3bed
2bath
1,244sqft
217-11 137th Ave, Springfield Gardens, NY 11413
For Sale
----------------------------
No Info
5bed
2bath
1,600sqft
31-32 99th St, East Elmhurst, NY 11369
For Sale
----------------------------
No Info
2bed
1bath
No Info
69-10 Yellowstone Blvd Unit 618, Forest Hills, NY 11375
For Sale
----------------------------
No Info
5bed
6bath
No Info
946 Saint Nicholas Ave, New York, NY 10032
For Sale
----------------------------
No Info
2bed
1bath
No Info
84-51 Beverly Rd Unit 2O, Kew Gardens, NY 11415
For Sale
----------------------------
No Info
2bed
1bath
No Info
21-06 35th St Unit 3C, Queens, NY 11105
For Sale
----------------------------
No Info
3bed
1bath
1,710sqft
201 Benton Ave, Staten Island, NY 10305
For Sale
----

LOOP START
Visiting the URL

Scraping page: 3
No Info
2bed
2bath
No Info
601 E 19th St Apt 6D, New York, NY 11226
For Sale
----------------------------
No Info
2bed
2bath
1,250sqft
34-25 150 Pl Unit 6B, Flushing, NY 11354
For Sale
----------------------------
No Info
4bed
2bath
1,600sqft
241 E 235th St, Bronx, NY 10470
For Sale
----------------------------
No Info
4bed
3bath
4,000sqft
121-01 153rd St, Jamaica, NY 11434
For Sale
----------------------------
No Info
2bed
2bath
1,195sqft
94-40 Magnolia Ct Unit 2A, Ozone Park, NY 11417
For Sale
----------------------------
No Info
2bed
1.5bath
2,415sqft
2525 Amboy Rd, Staten Island, NY 10306
For Sale
----------------------------
No Info
Studio
1bath
No Info
320 E 42nd St Apt 212, New York, NY 10017
For Sale
----------------------------
No Info
3bed
2.5bath
1,600sqft
966 Clove Rd Unit E, Staten Island, NY 10301
For Sale
----------------------------
No Info
2bed
1bath
900sqft
1744 E 55th St Unit 41C, Brooklyn, NY 11234
For Sale
-------------

In [14]:
pages

['/realestateandhomes-detail/1130-E-89th-St_Los-Angeles_CA_90002_M18168-39842',
 '/realestateandhomes-detail/2027-S-Ridgeley-Dr_Los-Angeles_CA_90016_M15350-13966',
 '/realestateandhomes-detail/18620-Hatteras-St-Unit-142_Tarzana_CA_91356_M13310-01304',
 '/realestateandhomes-detail/3518-W-3rd-St_Los-Angeles_CA_90020_M97802-20032',
 '/realestateandhomes-detail/3536-W-Slauson-Ave_Los-Angeles_CA_90043_M99049-32192',
 '/realestateandhomes-detail/9315-Burnet-Ave-Unit-112_North-Hills_CA_91343_M17484-24189',
 '/realestateandhomes-detail/1100-Wilshire-Blvd-Apt-2609_Los-Angeles_CA_90017_M15756-81809',
 '/realestateandhomes-detail/721-E-42nd-St_Los-Angeles_CA_90011_M26523-95986',
 '/realestateandhomes-detail/12105-Via-Santa-Marta_Sylmar_CA_91342_M14494-95965',
 '/realestateandhomes-detail/13216-S-Main-St_Los-Angeles_CA_90061_M14013-13211',
 '/realestateandhomes-detail/6301-S-Harcourt-Ave_Los-Angeles_CA_90043_M19562-10096',
 '/realestateandhomes-detail/4013-Guardia-Ave_Los-Angeles_CA_90032_M19774-1

In [ ]:
            # Extract property type and year property was built
            details = detail_page.find_all('li', attrs={"class": "rui-patterns__sc-2lxyoa-0 ivcDnD"})

            property_type = details[0].find('div', attrs={"class": "rui__ygf76n-0 gFfdWq rui__sc-17fo6pt-0 iRbBbO"})

            if property_type and property_type.text:
                types.append(property_type.text)
            else: 
                types.append('No Info')

            time.sleep(3)    
            browser.back()

In [17]:
df = pd.DataFrame({'Address': addresses, 'Status': status, 'Price': prices, 'Bed': beds, 'Bath': baths, 'Size': sizes})

# Extract address into Street, City, State, Zip
street_city = df['Address'].str.split(',', expand=True)
street_city = street_city.rename(columns={0: 'Street', 1: 'City', 2: 'state_zip'})
state_zip = street_city['state_zip'].str.split(' ', expand=True)
state_zip = state_zip.rename(columns={1: 'State', 2: 'Zip'})
street_city = street_city.drop(columns='state_zip')
state_zip = state_zip.drop(columns=0)

street_city.reset_index(drop=True, inplace=True)
state_zip.reset_index(drop=True, inplace=True)

# Create merged_df
address_df = pd.concat([street_city, state_zip], axis=1) 

merged_df = pd.concat([df, address_df], axis=1) 
merged_df

,Address,Status,Price,Bed,Bath,Size,Street,City,State,Zip
0,"1130 E 89th St, Los Angeles, CA 90002",For Sale,"$515,000",5bed,No Info,No Info,1130 E 89th St,Los Angeles,CA,90002
1,"2027 S Ridgeley Dr, Los Angeles, CA 90016",For Sale,"$2,500,000",12bed,No Info,"5,838sqft",2027 S Ridgeley Dr,Los Angeles,CA,90016
2,"18620 Hatteras St Unit 142, Tarzana, CA 91356",For Sale,"$340,000",2bed,2bath,910sqft,18620 Hatteras St Unit 142,Tarzana,CA,91356
3,"3518 W 3rd St, Los Angeles, CA 90020",For Sale,"$5,600,000",No Info,No Info,"32,548sqft",3518 W 3rd St,Los Angeles,CA,90020
4,"3536 W Slauson Ave, Los Angeles, CA 90043",For Sale,"$825,000",No Info,5bath,"2,300sqft",3536 W Slauson Ave,Los Angeles,CA,90043
...,...,...,...,...,...,...,...,...,...,...
541,"334 W 86th St Apt 5B, New York, NY 10024",For Sale,10.99,2bed,2bath,No Info,334 W 86th St Apt 5B,New York,NY,10024
542,"65-38 Austin St Unit 1A, Rego Park, NY 11374",For Sale,10.99,1bed,1bath,573sqft,65-38 Austin St Unit 1A,Rego Park,NY,11374
543,"50 Bridge Dr Unit 4A, Brooklyn, NY 11201",For Sale,10.99,4bed,3.5bath,"2,445sqft",50 Bridge Dr Unit 4A,Brooklyn,NY,11201
544,"750 Park Ave Unit 7BC, New York, NY 10021",For Sale,10.99,3bed,3.5bath,"2,100sqft",750 Park Ave Unit 7BC,New York,NY,10021


In [ ]:
browser.quit()